In [109]:
import pandas as pd

# Extracting data
df=pd.read_excel(r"E:\data analyst\Saeid\ALEX\Python\Data Cleaning in Pandas\Customer Call List.xlsx")

# Transforming

   # 1) Data Cleaning

#Removing duplicates
df=df.drop_duplicates()

#Removing redundant data(column)
df=df.drop(columns="Not_Useful_Column")

#Removing leading and trailing spaces (whitespace) from all columns
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#Removing  observed redundant characters
df['Last_Name']=df['Last_Name'].str.strip("123._/")

#Removing all non-word characters
#use \W to match any non-word character (equivalent to [^a-zA-Z0-9_]).
#df['Phone_Number'] = df['Phone_Number'].astype(str).str.replace('\W','')
df['Phone_Number']=df['Phone_Number'].str.replace('[^a-zA-Z0-9]','',regex=True)

#Converting vakues to string
df['Phone_Number'] = df['Phone_Number'].apply(lambda x: str(x))

#Styling Phone Numbers
df['Phone_Number'] = df['Phone_Number'].apply(lambda x: x[0:3] + '-' + x[3:6] + '-' + x[6:10])

#Replacing 'nan--' and 'Na--' with None (Removing them)
df['Phone_Number'] = df['Phone_Number'].replace('nan--','')
df['Phone_Number'] = df['Phone_Number'].replace('Na--','')

#Splitting Address to its components and substituting them in distinct columns
df[['Street_Address','State','Zip_Code']] = df["Address"].str.split(',', n=2, expand=True)

#Normalizing and unifying
df["Paying Customer"]=df["Paying Customer"].apply(lambda x: 'Yes' if x == 'Y' else ('No' if x == 'N' else x))
df["Do_Not_Contact"]=df["Do_Not_Contact"].apply(lambda x: 'Yes' if x == 'Y' else ('No' if x == 'N' else x))

#Replacing "N/a" then 'NA' with None (Removing them)
df=df.replace("N/a",'').fillna('')

   # 2) Data-Report Preparation

#Removing individuals from the list who should not be contacted
df2=df
for x in df2.index:
    if df2.loc[x, "Do_Not_Contact"] == 'Yes':
        df2.drop(x, inplace=True)
#Dropping individuals from the list if there is no phone number available for them      
#dropping null values
for x in df2.index:
    if df2.loc[x, "Phone_Number"] == '':
        df2.drop(x, inplace=True)
    

#Reseting index starting from 1 and name the index column
df3=df2
df3.reset_index(drop=True, inplace=True)
df3.index += 1
df3.rename_axis('Index', inplace=True)

# Loading

# Export DataFrames to Excel using ExcelWriter
with pd.ExcelWriter(r"E:\data analyst\Saeid\ALEX\Python\Data Cleaning in Pandas\Customer Call List Output.xlsx") as excel_writer:
    df.to_excel(excel_writer, sheet_name='Data Cleaning', index=False)
    df3.to_excel(excel_writer, sheet_name='Prepared Customer List To Call', index=False)

